In [1]:
from openai import OpenAI
import os
import re
from dotenv import load_dotenv, find_dotenv
load_dotenv('Key.env')
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [2]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo-1106", temperature=0.5):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    response_message = response.choices[0].message.content
    return response_message

In [3]:
def is_content_safe(content):
    blacklist = ['Child Porn', 'Sex', 'Minor', 'Porn']
    return all(word not in content.lower() for word in blacklist)

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    if not is_content_safe(prompt):
        context.append({'role': 'user', 'content': 'Moderation: Content rejected'})
        response = 'Sorry, your message was rejected due to inappropriate content.'
    else:
        context.append({'role': 'user', 'content': f"{prompt}"})
        response = get_completion_from_messages(context)

    context.append({'role': 'assistant', 'content': f"{response}"})
    user_pane = pn.pane.Markdown(prompt, width=600, styles={'color': 'red'})
    assistant_pane = pn.pane.Markdown(response, width=600, styles={'font-weight': 'bold'})

    panels.append(pn.Row('User:', user_pane))
    panels.append(pn.Row('Assistant:', assistant_pane))
    return pn.Column(*panels)

def close_input(event):
    inp.disabled = True

In [4]:
import panel as pn
pn.extension()

panels = []

context = [{'role': 'system', 'content': """
You are BookBot, an intelligent book recommendation assistant designed to help users /
discover new and exciting books based on their preferences. Your goal is to make reading /
recommendations that align with the user's interests. Start by greeting the user and asking /
about their preferred genres, authors, or any specific themes they enjoy. Consider factors /
like preferred book length, writing style, and tone.

Ask open-ended questions to gather information about their reading habits, such as whether /
they prefer fiction or non-fiction, historical or contemporary settings, and any favorite /
authors or genres. Be attentive to details and encourage users to express their preferences.

Once you have enough information, provide personalized book recommendations, including a /
brief description of each recommended book and why it might appeal to the user based on their /
preferences. Offer a variety of options to cater to different tastes.

If the user is uncertain or looking for a broader selection, you can suggest popular or /
critically acclaimed books in various genres. Always maintain a friendly and conversational /
tone throughout the interaction.

Remember, the key is to make the book recommendation process enjoyable and tailored to each /
user's unique reading preferences.

Please note that the output of the recommended books cannot be presented in a list format and cannot exceed 200 words/ 
Additionally, BookBot cannot provide recommendations for explicit or 18+ themes.
"""}]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
button_close = pn.widgets.Button(name="Close Chat")

interactive_conversation = pn.bind(collect_messages, button_conversation)

# Add the new button for closing the input field
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, button_close),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)
button_close.on_click(close_input)

dashboard.servable()


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
        [1] Button(name='Close Chat')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)